In [ ]:
import os
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('best.pt')

# Đường dẫn đến thư mục chứa các ảnh
image_folder = 'Images/data_thung_bia/images/val'
# Đường dẫn đến thư mục lưu kết quả
base_output_dir = 'data_thung_bia/output/val'

# Hàm để tạo thư mục nếu chưa tồn tại
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Tạo thư mục chính để lưu ảnh crop
create_dir_if_not_exists(base_output_dir)

# Duyệt qua từng file trong thư mục ảnh
for image_filename in os.listdir(image_folder):
    if image_filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
        image_path = os.path.join(image_folder, image_filename)
        
        # Đọc ảnh
        image = cv2.imread(image_path)
        
        # Chạy detection trên ảnh
        results = model(image, conf=0.2)
        
        # Lấy các thông tin của các bounding boxes
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy()
            scores = result.boxes.conf.cpu().numpy()
            
            for i, (box, cls, score) in enumerate(zip(boxes, classes, scores)):
                class_name = model.names[int(cls)]
                x1, y1, x2, y2 = map(int, box)
                
                # Tạo thư mục cho từng class
                class_dir = os.path.join(base_output_dir, class_name)
                create_dir_if_not_exists(class_dir)
                
                # Crop ảnh
                crop_img = image[y1:y2, x1:x2]
                
                # Đặt tên file với định dạng ảnh_tên_lớp_i.jpg
                existing_files = os.listdir(class_dir)
                next_index = len(existing_files) + 1
                crop_img_filename = os.path.join(class_dir, f"{os.path.splitext(image_filename)[0]}_{next_index}.jpg")
                
                # Lưu ảnh crop
                cv2.imwrite(crop_img_filename, crop_img)

print("Hoàn thành việc crop và lưu ảnh.")


In [24]:
from paddleocr import PaddleOCR, draw_ocr
paddle = PaddleOCR(use_angle_cls=False, lang="vi", use_gpu=False)

[2024/06/28 13:44:06] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/edabk-lab/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/edabk-lab/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_lengt

In [ ]:
# Đường dẫn đến ảnh đầu vào và thư mục đầu ra
image_path = 'data_thung_bia/output/val/billboard/text_det/66501431_1708602961854_71.jpg'
output_dir = 'data_thung_bia/output/val/billboard/text_det'
# Đọc ảnh
img = cv2.imread(image_path)
result = paddle.ocr(img, cls=True, det=True)

In [26]:
# Đường dẫn đến ảnh đầu vào và thư mục đầu ra
image_path = '/home/edabk-lab/khoinghiem/heniken/Hack-CMC-Heineken/models/advertise_recognize/data_thung_bia/output/val/billboard/66501431_1708602961854_71.jpg'
output_dir = 'data_thung_bia/output/val/billboard/text_det'

# Tạo thư mục đầu ra
create_dir_if_not_exists(output_dir)

# Đọc ảnh
img = cv2.imread(image_path)

# Phát hiện văn bản
result = paddle.ocr(img, cls=True, det=True)

[2024/06/28 13:47:32] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/06/28 13:47:32] ppocr DEBUG: dt_boxes num : 8, elapsed : 0.08135700225830078
[2024/06/28 13:47:32] ppocr DEBUG: rec_res num  : 8, elapsed : 0.288454532623291


In [29]:
import os
import cv2
import matplotlib.pyplot as plt
from paddleocr import PaddleOCR

# Hàm để tạo thư mục nếu chưa tồn tại
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Đường dẫn đến ảnh đầu vào và thư mục đầu ra
image_path = '/home/edabk-lab/khoinghiem/heniken/Hack-CMC-Heineken/models/advertise_recognize/data_thung_bia/output/val/billboard/66501431_1708602961854_71.jpg'
output_dir = 'data_thung_bia/output/val/billboard/text_det'

# Tạo thư mục đầu ra
create_dir_if_not_exists(output_dir)

# Đọc ảnh
img = cv2.imread(image_path)

# Phát hiện văn bản
result = paddle.ocr(img, cls=False, det=True)

if result and result[0]:
    result = result[0]
else:
    print("No text detected.")
    exit()

boxes = []
for line in result:
    line = line[0]
    boxes.append([[int(line[0][0]), int(line[0][1])], [int(line[2][0]), int(line[2][1])]])
boxes = boxes[::-1]

EXPEND = 5
w, h, _ = img.shape
for box in boxes:
    box[0][0] = max(0, box[0][0] - EXPEND)
    box[0][1] = max(0, box[0][1] - EXPEND)
    box[1][0] = min(w, box[1][0] + EXPEND)
    box[1][1] = min(h, box[1][1] + EXPEND)

print("Num text detected: ", len(boxes))

for box_index, box in enumerate(boxes):
    print("Box coordinates:", box)
    
    # Đảm bảo tọa độ hộp nằm trong phạm vi hợp lệ của ảnh
    if 0 <= box[0][1] < box[1][1] <= h and 0 <= box[0][0] < box[1][0] <= w:
        cropped_image = img[box[0][1]:box[1][1], box[0][0]:box[1][0]]
        crop_img_filename = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}_{box_index}.jpg")
        cv2.imwrite(crop_img_filename, cropped_image)
        print(f"Saved cropped image to {crop_img_filename}")
    else:
        print(f"Skipping invalid box: {box}")

print("Hoàn thành việc phát hiện văn bản và lưu ảnh.")

[2024/06/28 13:54:03] ppocr DEBUG: dt_boxes num : 8, elapsed : 0.03315234184265137
[2024/06/28 13:54:04] ppocr DEBUG: rec_res num  : 8, elapsed : 0.2229750156402588
Num text detected:  8
Box coordinates: [[382, 167], [222, 169]]
Skipping invalid box: [[382, 167], [222, 169]]
Box coordinates: [[476, 148], [222, 161]]
Skipping invalid box: [[476, 148], [222, 161]]
Box coordinates: [[434, 129], [222, 138]]
Skipping invalid box: [[434, 129], [222, 138]]
Box coordinates: [[46, 125], [222, 169]]
Saved cropped image to data_thung_bia/output/val/billboard/text_det/66501431_1708602961854_71_3.jpg
Box coordinates: [[408, 108], [222, 111]]
Skipping invalid box: [[408, 108], [222, 111]]
Box coordinates: [[262, 89], [222, 107]]
Skipping invalid box: [[262, 89], [222, 107]]
Box coordinates: [[147, 90], [204, 116]]
Saved cropped image to data_thung_bia/output/val/billboard/text_det/66501431_1708602961854_71_6.jpg
Box coordinates: [[362, 73], [222, 77]]
Skipping invalid box: [[362, 73], [222, 77]]
Hoà

In [39]:
import cv2
from paddleocr import PaddleOCR
import os

# Khởi tạo OCR
ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=False)  # Bạn có thể thay đổi 'lang' nếu cần thiết

# Thư mục chứa ảnh gốc và thư mục lưu ảnh crop
input_dir = 'data_thung_bia/output/val/billboard'
output_dir = 'data_thung_bia/output/val/billboard/text_det'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lấy danh sách các ảnh trong thư mục input
image_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]


for img_file in image_files:
    img_path = os.path.join(input_dir, img_file)
    img = cv2.imread(img_path)

    # Nhận dạng văn bản
    result = ocr.ocr(img_path, cls=True)

    # Kiểm tra nếu result không phải là None
    # Kiểm tra nếu result và phần tử đầu tiên của result không phải là None
    if result and len(result) > 0 and result[0]:
        for i, line in enumerate(result[0]):
            # line[0] là tọa độ bounding box, line[1] là văn bản và xác suất
            if line and len(line) > 0:
                coords = line[0]
                x1, y1 = int(coords[0][0]), int(coords[0][1])
                x2, y2 = int(coords[2][0]), int(coords[2][1])

                # Kiểm tra tọa độ có hợp lệ hay không
                if x1 < 0: x1 = 0
                if y1 < 0: y1 = 0
                if x2 > img.shape[1]: x2 = img.shape[1]
                if y2 > img.shape[0]: y2 = img.shape[0]

                # Crop ảnh theo tọa độ bounding box
                cropped_img = img[y1:y2, x1:x2]

                # Kiểm tra nếu cropped_img không rỗng
                if cropped_img.size > 0:
                    # Lưu ảnh đã crop
                    output_path = os.path.join(output_dir, f"{os.path.splitext(img_file)[0]}_{i}.jpg")
                    cv2.imwrite(output_path, cropped_img)

print("Hoàn thành việc crop và lưu ảnh.")


[2024/06/28 14:07:51] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/edabk-lab/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/edabk-lab/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, 